In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/drive/


In [13]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import torchvision.utils as vutils
import glob as glob
from tqdm import tqdm
import os
import cv2
import matplotlib


In [14]:
ROOT_PATH = 'C:/Users/Gaurav/Documents/Shaurya_project/simple_chairs'
print(len(os.listdir(ROOT_PATH)))

599


In [15]:
# Helper function to prepare the data from the Image Folders.

def prepare_dataset(ROOT_PATH):
    image_dirs = os.listdir(ROOT_PATH)
    image_dirs.sort()
    print(len(image_dirs))
    print(image_dirs[:5])

    all_image_paths = []

    for i in tqdm(range(len(image_dirs))):
        image_paths = glob.glob(f"{ROOT_PATH}/{image_dirs[i]}/*")
        image_paths.sort()
        for image_path in image_paths:
            all_image_paths.append(image_path)
        

    print(f"Total number of face images: {len(all_image_paths)}")

    train_data = all_image_paths[:-2000]
    valid_data = all_image_paths[-2000:]

    print(f"Total number of training image: {len(train_data)}")
    print(f"Total number of validation image: {len(valid_data)}")
    return train_data, valid_data

In [16]:
# this class prepares the image data to be fed into a nn.
class LFWDataset(Dataset):
    def __init__(self, data_list, transform):
        self.data = data_list
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = cv2.imread(self.data[index])
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image)
        return image

In [17]:
import imageio #for saving reconstruction images as GIF files.
from torchvision.utils import save_image

In [18]:
to_pil_image = transforms.ToPILImage()

#coverting images to tensors. 
def transform():
    transform = transforms.Compose([
            transforms.ToTensor(),
        ])
    return transform

In [19]:
# some helper functions for creating GIFs of reconstructed images and plotting the loss.
def image_to_vid(images):
    imgs = [np.array(to_pil_image(img)) for img in images]
    imageio.mimsave('./outputs/generated_images.gif', imgs)
def save_reconstructed_images(recon_images, epoch):
    save_image(recon_images.cpu(), f"./outputs/output{epoch}.jpg")
def save_loss_plot(train_loss, valid_loss):
    plt.figure(figsize=(10, 7))
    plt.plot(train_loss, color='orange', label='train loss')
    plt.plot(valid_loss, color='red', label='validataion loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('./outputs/loss.jpg')
    plt.show()

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
init_channels = 64 # initial number of filters
image_channels = 3 # color channels
latent_dim = 100 # number of features to consider
# define a Conv VAE
class ConvVAE(nn.Module):
    def __init__(self):
        super(ConvVAE, self).__init__()
 
        # encoder
        self.enc1 = nn.Conv2d(
            in_channels=image_channels, out_channels=init_channels, 
            kernel_size=4, stride=2, padding=2
        )
        self.enc2 = nn.Conv2d(
            in_channels=init_channels, out_channels=init_channels*2, 
            kernel_size=4, stride=2, padding=2
        )
        self.enc3 = nn.Conv2d(
            in_channels=init_channels*2, out_channels=init_channels*4, 
            kernel_size=4, stride=2, padding=2
        )
        self.enc4 = nn.Conv2d(
            in_channels=init_channels*4, out_channels=init_channels*8, 
            kernel_size=4, stride=2, padding=2
        )
        self.enc5 = nn.Conv2d(
            in_channels=init_channels*8, out_channels=1024, 
            kernel_size=4, stride=2, padding=2
        )
        self.fc1 = nn.Linear(1024, 2048)
        self.fc_mu = nn.Linear(2048, latent_dim)
        self.fc_log_var = nn.Linear(2048, latent_dim)
        self.fc2 = nn.Linear(latent_dim, 1024)
        # decoder 
        self.dec1 = nn.ConvTranspose2d(
            in_channels=1024, out_channels=init_channels*8, 
            kernel_size=3, stride=2
        )
        self.dec2 = nn.ConvTranspose2d(
            in_channels=init_channels*8, out_channels=init_channels*4, 
            kernel_size=3, stride=2
        )
        self.dec3 = nn.ConvTranspose2d(
            in_channels=init_channels*4, out_channels=init_channels*2, 
            kernel_size=3, stride=2
        )
        self.dec4 = nn.ConvTranspose2d(
            in_channels=init_channels*2, out_channels=init_channels, 
            kernel_size=3, stride=2
        )
        self.dec5 = nn.ConvTranspose2d(
            in_channels=init_channels, out_channels=image_channels, 
            kernel_size=4, stride=2
        )
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling
        return sample
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))
        batch, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        hidden = self.fc1(x)
        # get `mu` and `log_var`
        mu = self.fc_mu(hidden)
        log_var = self.fc_log_var(hidden)
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
        z = self.fc2(z)
        z = z.view(-1, 1024, 1, 1)
 
        # decoding
        x = F.relu(self.dec1(z))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        reconstruction = torch.sigmoid(self.dec5(x))
        return reconstruction, mu, log_var

In [21]:
#The loss function
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)

    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD 

In [22]:
# the training function
def train(model, dataloader, dataset, device, optimizer, criterion):
    model.train()
    running_loss = 0.0
    counter = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(dataset)/dataloader.batch_size)):
        counter += 1
        data = data
        data = data.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    train_loss = running_loss / counter 
    return train_loss

In [23]:
# The validation function.
def validate(model, dataloader, dataset, device, criterion):
    model.eval()
    running_loss = 0.0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(dataset)/dataloader.batch_size)):
            counter += 1
            data= data
            data = data.to(device)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            if i == int(len(dataset)/dataloader.batch_size) - 1:
                recon_images = reconstruction
    val_loss = running_loss / counter
    return val_loss, recon_images

In [24]:
matplotlib.style.use('ggplot')
# define the computation device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# a list to save all the reconstructed images in PyTorch grid format
grid_images = []
device

device(type='cuda')

In [25]:
# initialize the model
model = ConvVAE().to(device)
# define the learning parameters
lr = 0.0001
epochs = 75
batch_size = 64
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss(reduction='sum')

In [26]:
# initialize the transform
transform = transform()
# prepare the training and validation data loaders
train_data, valid_data = prepare_dataset(ROOT_PATH)
trainset = LFWDataset(train_data, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
validset = LFWDataset(valid_data, transform=transform)
validloader = DataLoader(validset, batch_size=batch_size)

599
['117930a8f2e37f9b707cdefe012d0353', '11c4b7e1843657b2ee93c6b133692f1b', '11d3fc4092e616a7a6fee8e2140acec9', '11d9817e65d7ead6b87028a4b477349f', '12838414141e59219a2d92ec05f8de08']


100%|███████████████████████████████████████████████████████████████████████████████| 599/599 [00:01<00:00, 383.59it/s]

Total number of face images: 37138
Total number of training image: 35138
Total number of validation image: 2000


In [32]:
train_loss = []
valid_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = train(
        model, trainloader, trainset, device, optimizer, criterion
    )
    valid_epoch_loss, recon_images = validate(
        model, validloader, validset, device, criterion
    )
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    # save the reconstructed images from the validation loop
    save_reconstructed_images(recon_images, epoch+1)
    # convert the reconstructed images to PyTorch image grid format
    image_grid = make_grid(recon_images.detach().cpu())
    grid_images.append(image_grid)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {valid_epoch_loss:.4f}")

Epoch 1 of 75


550it [02:09,  4.25it/s]                                                                                               
32it [00:07,  4.40it/s]                                                                                                


Train Loss: 90665.2573
Val Loss: 57595.9276
Epoch 2 of 75


550it [02:06,  4.34it/s]                                                                                               
32it [00:06,  4.73it/s]                                                                                                


Train Loss: 55212.7790
Val Loss: 53738.6167
Epoch 3 of 75


...

Train Loss: 24510.0174
Val Loss: 27842.3916
Epoch 74 of 75


550it [02:05,  4.39it/s]                                                                                               
32it [00:06,  4.69it/s]                                                                                                


Train Loss: 24458.6239
Val Loss: 27834.9159
Epoch 75 of 75


550it [02:06,  4.36it/s]                                                                                               
32it [00:06,  4.72it/s]                                                                                                

Train Loss: 24409.3778
Val Loss: 28076.7893


In [11]:
# Generating the video

images = []
files = glob.glob ("./outputs/*.jpg")
for myFile in files:
    print(myFile)
    image = cv2.imread (myFile)
    images.append(image)

image_to_vid(images)

SyntaxError: invalid non-printable character U+00A0 (1984324086.py, line 3)

In [12]:
# plotting the loss

save_loss_plot(train_loss, valid_loss)

SyntaxError: invalid non-printable character U+00A0 (688244474.py, line 3)

In [ ]:
# generating images on new instances.

# TEST_ROOT_PATH = 'test_path'
# test_data = prepare_dataset(TEST_ROOT_PATH)
# testset = LFWDataset(test_data, transform=transform)
# testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)



for data in trainloader:
  rec, _, _ = model(data)
  save_test_reconstructed_images()
  print(rec.shape)
  plt.imshow(
  break